In [ ]:
import sys,os

RES_PATH = 'meshes' 

if not os.path.exists(RES_PATH):
    print( 'cannot find meshes  please update RES_PATH')
    exit(1)
else:
    pass

import scipy as sp
import numpy as np
import meshplot as mp
import trimesh
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from os import listdir
from os.path import isfile, join
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
import igl

In [ ]:
def shade_by_color(mesh, color):
    if len(color) == len(mesh.vertices):
        shading = {"width": 300, "height": 300}
        return mp.plot(mesh.vertices, mesh.faces, shading=shading, c=color, return_plot=True)
    else:
        shading = {"width": 300, "height": 300}
        return mp.plot(mesh.vertices, mesh.faces, shading=shading, return_plot=True)

In [ ]:
def find_boundary_vertices(mm):
    # code from https://github.com/mikedh/trimesh/issues/1188
    unique_edges = mm.edges[trimesh.grouping.group_rows(mm.edges_sorted, require_count=1)]
    boundary_vertices = np.unique(unique_edges.flatten())
    boundary_vertex_coord = mm.vertices[boundary_vertices]
    return boundary_vertices, boundary_vertex_coord

In [ ]:
def compute_lap_cot(mm, dirichlet=0):
    num_vertices = mm.vertices.shape[0]
    # create empty matrix to store laplace operator
    C = np.zeros([num_vertices, num_vertices])
    C = lil_matrix(C)
    M = np.zeros(num_vertices)
    boundary_vertices = 0

    if dirichlet:
        boundary_vertices, _ = find_boundary_vertices(mm)

    # loop through all vertices
    for i in range(num_vertices):
        # get the neighbors of current vertex
        neighbors = mm.vertex_neighbors[i]
        # for each neighbor
        for neighbor in neighbors:
            angles = []
            # find faces that contains the edge between vertex & current neighbor
            # for each face
            for face in mm.vertex_faces[i]:
                # if not an empty entry
                if face != -1:
                    face_vertices = mm.faces[face]
                    # if face share this edge
                    if neighbor in face_vertices:
                        # get angle of the third vertex
                        # first find the index of the third vertex
                        dup = list(face_vertices).copy()
                        dummy_idx = [0, 1, 2]
                        # removes the index of the current neighbor and vertex
                        # the index remaining is the vertex we are looking for
                        dummy_idx.remove(dup.index(neighbor))
                        dummy_idx.remove(dup.index(i))
                        assert len(dummy_idx) == 1
                        # get the angle
                        angles.append(mm.face_angles[face, dummy_idx[0]])
                        
            # get sum of cotans 
            # remove 0s if there are any
            angles = [value for value in angles if value != 0]
            angles = np.array(angles)
            cot_angles_sum = np.sum(1 / np.tan(angles)) / 2

            # construct C matrix
            C[i, neighbor] = cot_angles_sum
            C[i, i] -= cot_angles_sum  

            if dirichlet:
                if neighbor in boundary_vertices:
                    C[i, neighbor] = 0

        # for every connected face
        for face in mm.vertex_faces[i]:
            # if not an empty entry
            if face != -1:
                # sum up triangle areas connected to the current vertex
                M[i] += mm.area_faces[face]
                
    # compute laplace beltrami operator
    M = M / 3
    M_inv = sp.sparse.spdiags([1 / M], np.array([0]))
    C = C.tocsc()
    L = M_inv @ C

    return L, M, C

In [ ]:
def compute_heat_flow(Lc, A, sources, t):
    k_delta = np.zeros(len(A))
    k_delta[sources] = 1
    A = sp.sparse.spdiags([A], np.array([0]))
    u = sp.sparse.linalg.spsolve(A - t * Lc, k_delta)
    return u

In [ ]:
def compute_gradient(mm, u, edges):
    grad = np.zeros([len(mm.faces), 3])
    for i in range(len(mm.faces)):
        face = mm.faces[i]
        delta_u_0 = u[face[0]] * np.cross(mm.face_normals[i], edges[i, 1, :])
        delta_u_1 = u[face[1]] * np.cross(mm.face_normals[i], edges[i, 2, :])
        delta_u_2 = u[face[2]] * np.cross(mm.face_normals[i], edges[i, 0, :])
        delta_u = (1 / (2 * mm.area_faces[i])) * (delta_u_0 + delta_u_1 + delta_u_2)
        grad[i, :] = delta_u
    return grad

In [ ]:
def compute_edges(mm):
    edges = []
    for face in mm.faces:
        # counter clockwise
        e0 = mm.vertices[face[1]] - mm.vertices[face[0]]
        e1 = mm.vertices[face[2]] - mm.vertices[face[1]]
        e2 = mm.vertices[face[0]] - mm.vertices[face[2]]
        edges.append([e0, e1, e2])

    return np.array(edges)

In [ ]:
def cotan(angle):
    cot = 1 / np.tan(angle)
    return cot

In [ ]:
def compute_divergence(mm, X):
    divergence = np.zeros(len(mm.vertices))
    for i in range(len(mm.faces)):
        face = mm.faces[i]
        face_angles = mm.face_angles[i]
        verts = mm.vertices[face]
        divergence[face[0]] += cotan(face_angles[2]) * np.dot((verts[1] - verts[0]), X[i]) + cotan(face_angles[1]) * np.dot((verts[2] - verts[0]), X[i])
        divergence[face[1]] += cotan(face_angles[2]) * np.dot((verts[0] - verts[1]), X[i]) + cotan(face_angles[0]) * np.dot((verts[2] - verts[1]), X[i])
        divergence[face[2]] += cotan(face_angles[1]) * np.dot((verts[0] - verts[2]), X[i]) + cotan(face_angles[0]) * np.dot((verts[1] - verts[2]), X[i])
    divergence = divergence / 2
    return divergence

In [ ]:
def compute_time(mm, m):
    h = np.mean(mm.edges_unique_length)
    t = m * h ** 2
    return t

In [ ]:
def add_isolines(mm, plot, geodesics, num_lines):
    isov, isoe = igl.isolines(mm.vertices, mm.faces, geodesics, num_lines)
    plot.add_lines(isov[isoe][:, 0, :], isov[isoe][:, 1, :], shading={"line_color": "white"})

In [ ]:
def visualize_gradient(mm, X):
    centroids = np.zeros((mm.faces.shape[0], 3))
    for i in range(mm.faces.shape[0]):
        centroid = np.mean(mm.vertices[mm.faces[i]], axis=0)
        centroids[i, :] = centroid

    p = shade_by_color(mm, [])
    p.add_lines(centroids, centroids + X*0.005, shading={"line_color": "red"})
    

In [ ]:
def compute_geodesics(mm, Lc, A, sources):
    t = compute_time(mm, 1)
    u = compute_heat_flow(Lc, A, sources, t)
    edges = compute_edges(mm)
    grad = compute_gradient(mm, u, edges)
    X = (- grad.T / np.linalg.norm(grad, axis=1)).T
    divergence = compute_divergence(mm, X)
    geodesics = sp.sparse.linalg.spsolve(Lc, divergence)

    return u, X, geodesics

In [ ]:
def compute_geodesics_mixed(mm, Lc_D, Lc_N, A, sources):
    t = compute_time(mm, 1)
    u_D = compute_heat_flow(Lc_D, A, sources, t)
    u_N = compute_heat_flow(Lc_N, A, sources, t)
    u = (u_D + u_N) / 2
    edges = compute_edges(mm)
    grad = compute_gradient(mm, u, edges)
    X = (- grad.T / np.linalg.norm(grad, axis=1)).T
    divergence = compute_divergence(mm, X)
    geodesics = sp.sparse.linalg.spsolve(Lc_N, divergence)

    return u, X, geodesics


In [ ]:
def convert_colors(geodesics):
    # normalize to a color map
    norm = Normalize(vmin=np.min(geodesics), vmax=np.max(geodesics))
    cmap = cm.get_cmap('gist_heat_r')
    colors = cmap(norm(geodesics))[:, :3]

    return colors

# Geodesic Distance

In [ ]:
# load mesh
mesh = os.path.join(RES_PATH,'bunny.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

L, A, Lc = compute_lap_cot(mm)


In [ ]:
sources = [0]

u, X, geodesics = compute_geodesics(mm, Lc, A, sources)
colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
add_isolines(mm, p, geodesics, 30)

In [ ]:
# plot heat flow
p = shade_by_color(mm, u)

In [ ]:
# plot gradient
visualize_gradient(mm, X)

# Boundary Conditions

In [ ]:
# load mesh
mesh = os.path.join(RES_PATH,'sphere_small.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

L, A, Lc = compute_lap_cot(mm, dirichlet=0)

sources = [80]

u, X, geodesics = compute_geodesics(mm, Lc, A, sources)
colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
add_isolines(mm, p, geodesics, 30)

In [ ]:
# load mesh
mesh = os.path.join(RES_PATH,'half_sphere_small.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

L, A, Lc = compute_lap_cot(mm, dirichlet=0)

sources = [80]

u, X, geodesics = compute_geodesics(mm, Lc, A, sources)
colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
add_isolines(mm, p, geodesics, 30)

In [ ]:
L, A, Lc = compute_lap_cot(mm, dirichlet=1)

sources = [80]

u, X, geodesics = compute_geodesics(mm, Lc, A, sources)
colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
add_isolines(mm, p, geodesics, 25)


In [ ]:
mesh = os.path.join(RES_PATH,'half_sphere_small.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

L, A, Lc_N = compute_lap_cot(mm, dirichlet=0)
L, A, Lc_D = compute_lap_cot(mm, dirichlet=1)

sources = [80]

u, X, geodesics = compute_geodesics_mixed(mm, Lc_D, Lc_N, A, sources)
colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
add_isolines(mm, p, geodesics, 40)


# Path Finding

In [ ]:
def normalize(geodesics):
    geodesics = (geodesics - np.min(geodesics)) / (np.max(geodesics) - np.min(geodesics))
    return geodesics

In [ ]:
def trace_shortest_path(mm, source, destination):
    curr_vertex = source
    L, A, Lc = compute_lap_cot(mm)
    u, X, geodesics = compute_geodesics(mm, Lc, A, destination)
    geodesics = normalize(geodesics)
    path = [source]

    while curr_vertex != destination:
        neighbors = mm.vertex_neighbors[curr_vertex]
        closest_neighbor = neighbors[np.argmin(geodesics[neighbors])]
        curr_vertex = closest_neighbor
        path.append(curr_vertex)

    return geodesics, path

In [ ]:
def plot_path(plot, path):
    plot.add_lines(mm.vertices[path[:-1]], mm.vertices[path[1:]], shading={"line_color": "blue"})

In [ ]:
# load mesh
mesh = os.path.join(RES_PATH,'bunny.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

geodesics, path = trace_shortest_path(mm, 0, 100)

colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
add_isolines(mm, p, geodesics, 35)
plot_path(p, path)

In [ ]:
# terrain generated by https://touchterrain.geol.iastate.edu/
# stl file edited in meshlab for cleaning and simplification
mesh = os.path.join(RES_PATH,'topograph.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

geodesics, path = trace_shortest_path(mm, 200, 2000)
colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
# add_isolines(mm, p, geodesics, 30)
plot_path(p, path)


In [ ]:
mesh = os.path.join(RES_PATH,'terrain.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

sources = [200, 2000]

geodesics, path = trace_shortest_path(mm, 200, 1000)
colors = convert_colors(geodesics)
p = shade_by_color(mm, colors)
# add_isolines(mm, p, geodesics, 30)
plot_path(p, path)


# Noisy Data

In [ ]:
def add_noise(k, mesh):
    # make a duplicate mesh
    dup = mesh.copy()
    vertices = mesh.vertices
    num_vertices = vertices.shape[0]

    # compute sigma based on the size of the bounding box in each dimension
    sigma_x = np.abs(np.min(vertices[:, 0]) - np.max(vertices[:, 0])) * k
    sigma_y = np.abs(np.min(vertices[:, 1]) - np.max(vertices[:, 1])) * k
    sigma_z = np.abs(np.min(vertices[:, 2]) - np.max(vertices[:, 2])) * k

    # generate 0 mean gaussian noise with the sigma
    x_noise = np.random.normal(0, sigma_x, num_vertices)
    y_noise = np.random.normal(0, sigma_y, num_vertices)
    z_noise = np.random.normal(0, sigma_z, num_vertices)
    # add the generated noise to vertex positions
    noise = np.stack((x_noise, y_noise, z_noise)).reshape(num_vertices, 3)
    
    # add noise
    noisy_vertices = vertices + noise
    dup.vertices = noisy_vertices

    # return new noisy mesh
    return dup

In [ ]:
mesh = os.path.join(RES_PATH,'bunny.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh)
noisy = add_noise(0.005, mm)

L, A, Lc = compute_lap_cot(noisy)

sources = [0]

u, X, geodesics = compute_geodesics(noisy, Lc, A, sources)
colors = convert_colors(geodesics)
p = shade_by_color(noisy, colors)
add_isolines(noisy, p, geodesics, 30)

In [ ]:
mesh = os.path.join(RES_PATH,'bunny.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh)
noisy = add_noise(0.01, mm)

L, A, Lc = compute_lap_cot(noisy)

sources = [0]

u, X, geodesics = compute_geodesics(noisy, Lc, A, sources)
colors = convert_colors(geodesics)
p = shade_by_color(noisy, colors)
add_isolines(noisy, p, geodesics, 30)